# SerpAPI Tool Helper

This notebook provides a quick rundown of how we construct an async SerpAPI tool. Let's start by query SerpAPI synchronously via their Python SDK:

In [3]:
from serpapi import GoogleSearch
from getpass import getpass

SERPAPI_API_KEY = getpass("Enter your SerpAPI API key: ")

params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

search = GoogleSearch(params)
results = search.get_dict()

print(results)

{'search_metadata': {'id': '683439458da5ea4a83599e15', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/d44066da4ed52755/683439458da5ea4a83599e15.json', 'created_at': '2025-05-26 09:49:57 UTC', 'processed_at': '2025-05-26 09:49:57 UTC', 'google_url': 'https://www.google.com/search?q=latest+news+in+the+world&oq=latest+news+in+the+world&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/d44066da4ed52755/683439458da5ea4a83599e15.html', 'total_time_taken': 0.96}, 'search_parameters': {'engine': 'google', 'q': 'latest news in the world', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'latest news in the world', 'total_results': 6160000000, 'time_taken_displayed': 0.21, 'organic_results_state': 'Results for exact spelling'}, 'related_questions': [{'question': 'What is the trending news in the world now?', 'snippet': None, 'title': 'World', 'link': 'https://www.cnn.com/world', 'list': ['Head of controver

Our results are provided in the `"organic_results"` key:

In [4]:
results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwi5xZz37cCNAxXdl4kEHbGvEuAQFnoECFgQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/683439458da5ea4a83599e15/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': "World · 'He has gone absolutely crazy': Trump rebukes Putin as Russia hits Ukraine with record drone attack · Israeli strikes across Gaza kill 54, officials say.",
  'snippet_highlighted_words': ['Trump rebukes Putin as Russia hits Ukraine with record drone attack'],
  'sitelinks': {'inline': [{'title': 'BBC World News',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Africa', 'link': 'https://www.bbc.com/news/world/africa'},
    {'title': 'Middle East',
     'link': 'https://w

We reformat this to extract only the most relevant information, such as the the title, source, link, and snippet. Let's use pydantic `BaseModel` to define this structure.

In [5]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    source: str
    link: str
    snippet: str

    @classmethod
    def from_serpapi_result(cls, result: dict) -> "Article":
        return cls(
            title=result["title"],
            source=result["source"],
            link=result["link"],
            snippet=result["snippet"],
        )

In [6]:
articles = [Article.from_serpapi_result(result) for result in results["organic_results"]]

## Using Async

All of this works, but it unfortunately is not async and the SerpAPI SDK does not support async either, so we much query the API directly using the `aiohttp` library.

In [7]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get(
        "https://serpapi.com/search",
        params=params
    ) as response:
        results = await response.json()

results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwi5xZz37cCNAxXdl4kEHbGvEuAQFnoECFgQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/683439458da5ea4a83599e15/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': "World · 'He has gone absolutely crazy': Trump rebukes Putin as Russia hits Ukraine with record drone attack · Israeli strikes across Gaza kill 54, officials say.",
  'snippet_highlighted_words': ['Trump rebukes Putin as Russia hits Ukraine with record drone attack'],
  'sitelinks': {'inline': [{'title': 'BBC World News',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Africa', 'link': 'https://www.bbc.com/news/world/africa'},
    {'title': 'Middle East',
     'link': 'https://w

With that, we have all we need to build a fully async serpapi tool.

In [8]:
from langchain_core.tools import tool

@tool
async def serpapi(query: str) -> list[Article]:
    """Use this tool to search the web."""
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": query,
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://serpapi.com/search",
            params=params
        ) as response:
            results = await response.json()
    return [Article.from_serpapi_result(result) for result in results["organic_results"]]

Note that because this tool is async, we cannot use `tool.func` to call it as before:

In [9]:
serpapi.func

Instead, we use `tool.coroutine`:

In [10]:
serpapi.coroutine

<function __main__.serpapi(query: str) -> list[__main__.Article]>

---